In [1]:
import os
from r2r import R2RClient
from scraper import Scraper
from splitter import Splitter
from r2r_backend import R2RBackend
from helper.io_helper import iterate_over_files, load_prompt

scraper = Scraper()
splitter = Splitter()
r2r_backend = R2RBackend(R2RClient(os.getenv("R2R_HOSTNAME", "http://localhost:7272")))

2024-10-21 23:47:38,951 - WARNING - langchain_community.utils.user_agent - USER_AGENT environment variable not set, consider setting it to identify your requests.


NameError: name 'Path' is not defined

In [2]:
from r2r import 

NameError: name 'r2r_backend' is not defined

In [6]:
r2r_backend.health()

{'response': 'ok'}

In [7]:
files = list(iterate_over_files("files"))
ingestion_resp = r2r_backend.ingest_files(files)

2024-10-21 23:28:33,605 - ERROR - r2r_backend - Error during ingestion: Must increment version number before attempting to overwrite document b0e9ab39-a800-552d-a31c-d1c3f7c19968. Use the `update_files` endpoint if you are looking to update the existing version.


Exception: Error during ingestion: Must increment version number before attempting to overwrite document b0e9ab39-a800-552d-a31c-d1c3f7c19968. Use the `update_files` endpoint if you are looking to update the existing version.

In [3]:
urls = ["https://en.wikipedia.org/wiki/Bitcoin", "https://en.wikipedia.org/wiki/Dogecoin"]

In [7]:
documents = scraper.fetch_documents(urls)
for document in documents:
    split_document = splitter.split_documents([document])
    chunks = [{"text": chunk.page_content} for chunk in split_document]
    metadata = document.metadata
    r2r_backend.ingest_chunks(chunks, metadata)

In [12]:
doc_id = r2r_backend.documents_overview()[0]['id']
chunks = r2r_backend.document_chunks(doc_id)

'^ Sigalos, MacKenzie (October 27, 2022). "Dogecoin surges 35% as Elon Musk\'s Twitter deal approaches close". CNBC. Archived from the original on October 28, 2023. Retrieved October 28, 2023.\n^ Leffer, Lauren (April 7, 2023). "Doge\'s Twitter Reign Ends, Triggering Memecoin Slump". Gizmodo. Archived from the original on April 8, 2023. Retrieved April 8, 2023.'

In [2]:
query = input("Enter your query: ")
rag_rsp = r2r_backend.rag(query)
print(rag_rsp['completion']['choices'][0]['message']['content'])

Based on the provided context, here's a response to the query:

Blockchain is a decentralized digital ledger that records transactions across a peer-to-peer network. It's implemented as an ordered list of blocks, each containing a SHA-256 hash of the previous block, chaining them in chronological order [2]. The blockchain is maintained by a peer-to-peer network, and individual blocks, public addresses, and transactions within blocks are public information that can be examined using a blockchain explorer [3].

In the context of Bitcoin, the blockchain is used to record all bitcoin transactions. Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in the blockchain without central oversight [1]. The consensus between nodes is achieved using a computationally intensive process based on proof of work, called mining, that secures the bitcoin blockchain [1].

The "Princeton" definition of blockchain describes it as a linked list built with hash p

In [7]:
from pathlib import Path
prompt_name = 'custom_rag_template'
prompt_data = load_prompt(Path('chat_templates'), prompt_name)
template = prompt_data['template']
input_types = prompt_data['input_types']

r2r_backend.add_prompt(prompt_name, template, input_types)

{'message': "Prompt 'custom_rag_template' added successfully."}

In [14]:
r2r_backend.get_all_prompts()['prompts']['custom_rag_template']

{'name': 'custom_rag_template',
 'template': '## Task: You are an AI assistant specialized in providing accurate and context-based responses. Your task is to answer user queries using only the information from the provided context. Your responses should be factual, comprehensive, and directly reference the given context.\n## Context: {context}\n## Query: {query}\n## Instructions: 1. Analyze the query and the provided context carefully. 2. Formulate a response based solely on the information in the context. 3. Use specific references (e.g., [1], [2]) to cite relevant parts of the context in your response. 4. Ensure your answer is as comprehensive as possible within the constraints of the given context. 5. If the query cannot be fully answered from the context:\n  a. Provide partial information if available, clearly stating what aspects you can address.\n  b. Indicate which parts of the query cannot be answered based on the given context.\n6. If the query is entirely outside the scope of